In [ ]:
# This jupyter notebook was developed in Colab.
# The train and test set were upload to my Drive and were fetched directly from Google Drive
# Current Directory in uses:
# - training_path = '/content/drive/My Drive/Colab Notebooks/5_shot/train'
# - testing_path = '/content/drive/My Drive/Colab Notebooks/5_shot/test'



# To run script:
# 1. update the training_path and testing_path (line 17 and 18) to where the dataset is located
# 2. Run all the cells sequencially


# Set path to train/test dataset (i.e., images)
from google.colab import drive

drive.mount('/content/drive')
training_path = '/content/drive/My Drive/Colab Notebooks/5_shot/train'
testing_path = '/content/drive/My Drive/Colab Notebooks/5_shot/test'

In [ ]:
### Load Data and split training dataset into training and validation sets
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

 # Low batch size for faster processing
batch_size = 32
# Defined Image size
img_height = 530
img_width = 1020
# Defined class (integer label type)
class_names = list(range(0,22))
for i in range(len(class_names)):
    class_names[i] = str(class_names[i])
label_type = 'int'

# Split the trianning set into: 20% validation, 80% training
# Define parameters for the sets
training_set = tf.keras.utils.image_dataset_from_directory(
    training_path,
    subset="training",
    validation_split=0.2,
    label_mode=label_type,
    class_names=class_names,
    seed=123, # Randomize the set to prevent overfitting by sequence
    image_size=(img_height, img_width),
    batch_size=batch_size
    )

validation_set = tf.keras.utils.image_dataset_from_directory(
    training_path,
    subset="validation",
    validation_split=0.2,
    label_mode=label_type,
    class_names=class_names,
    seed=123,  # Randomize the set to prevent overfitting by sequence
    image_size=(img_height, img_width),
    batch_size=batch_size
    )

# Set autotune function to automatically tune parameters dynamically at runtime
AT = tf.data.AUTOTUNE
training_set = training_set.cache().shuffle(1000).prefetch(buffer_size=AT) # Shuffle to reduce the likelihood of overfitting
validation_set = validation_set.cache().prefetch(buffer_size=AT)

Found 110 files belonging to 22 classes.
Using 88 files for training.
Found 110 files belonging to 22 classes.
Using 22 files for validation.
(32, 530, 1020, 3) (32,)


In [ ]:
### Load base model to be trained
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# Initiate a base model based on VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(530, 1020, 3))
len(base_model.layers)

# Build a training model
model = tf.keras.models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(22, activation='softmax')  # Assuming 22 classes
])

In [ ]:
# Freeze the base model's pre-trained layers
base_model.trainable = False

# Re-compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)


# Train the model
epochs_initial = 10
history_train = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=epochs_initial
    callbacks=[early_stopping, model_checkpoint]
)

# # Fine-tuning
# base_model.trainable = True
# fine_tune_at = 9 * len(base_model.layers) // 10

# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False

# # Use a smaller learning rate for fine-tuning
# optimizer_fine = tf.keras.optimizers.Adam(learning_rate=1e-5)
# model.compile(optimizer=optimizer_fine, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Fine-tune the model
# epochs_fine_tune = 2
# history_finetune = model.fit(
#     training_set,
#     validation_data=validation_set,
#     epochs=epochs_fine_tune,
#     callbacks=[early_stopping, model_checkpoint]
# )


Epoch 1/10
3/3 [==============================] - 513s 167s/step - loss: 48.7940 - accuracy: 0.0341 - val_loss: 18.4062 - val_accuracy: 0.0455
Epoch 2/10
3/3 [==============================] - 497s 164s/step - loss: 17.6468 - accuracy: 0.0341 - val_loss: 4.2802 - val_accuracy: 0.0909
Epoch 3/10
3/3 [==============================] - 493s 163s/step - loss: 3.9990 - accuracy: 0.0795 - val_loss: 3.2515 - val_accuracy: 0.0909
Epoch 4/10
3/3 [==============================] - 496s 185s/step - loss: 3.1367 - accuracy: 0.0455 - val_loss: 3.1810 - val_accuracy: 0.0000e+00
Epoch 5/10
3/3 [==============================] - 494s 163s/step - loss: 3.0933 - accuracy: 0.0682 - val_loss: 3.1951 - val_accuracy: 0.0455
Epoch 6/10
3/3 [==============================] - 493s 163s/step - loss: 3.0815 - accuracy: 0.0568 - val_loss: 3.1310 - val_accuracy: 0.0455
Epoch 7/10
3/3 [==============================] - 494s 163s/step - loss: 3.1224 - accuracy: 0.0341 - val_loss: 3.1705 - val_accuracy: 0.0000e+00
Ep

In [ ]:
### Load test data and predict
import numpy as np
import pandas as pd
test_ds = tf.keras.utils.image_dataset_from_directory(
    testing_path,
    image_size=(530, 1020),
    batch_size=batch_size,
    shuffle=False,
    label_mode=None  # Test Data isn't labelled
)

predictions = model.predict(test_ds)
predicted_classes = np.argmax(predictions, axis=1)

# Complete file paths
test_filenames = test_ds.file_paths
# Extract filenames without extension
for i in range(len(test_filenames)):
    path = test_filenames[i]
    filename_without_extension = path.split('/')[-1].split('.')[0]
    test_filenames[i] = filename_without_extension

# Create a Excle Sheet for submission
submission = pd.DataFrame({'ID': test_filenames, 'Category': predicted_classes})
submission.to_csv('submission.csv', index=False)


Found 517 files belonging to 1 classes.
17/17 [==============================] - 815s 48s/step


In [ ]:
# Install packages
import numpy as np
import networkx as nx
! pip install gurobipy
from gurobipy import Model, GRB
import gurobipy as gp
import matplotlib.pyplot as plt
import pandas as pd

# Setup data
np.random.seed(2344)
N = np.random.randint(20, 26)

A1 = np.array([np.random.permutation(N) for _ in range(N)])
A2 = np.random.choice([0, 1], size=(N, N), p=[0.5, 0.5])

df_A1 = pd.DataFrame(A1)
print(df_A1)
df_A2 = pd.DataFrame(A2)
print(df_A2)

# Part 1 1A
model_min = Model()
vars_min = model_min.addVars(N, N, vtype=GRB.BINARY)
model_min.setObjective(sum(vars_min[i, j] * A1[i, j] for i in range(N) for j in range(N)), GRB.MINIMIZE)

for i in range(N):
  model_min.addConstr(sum(vars_min[i, j] for j in range(N)) == 1)
  model_min.addConstr(sum(vars_min[j, i] for j in range(N)) == 1)

model_min.optimize()

solution = np.zeros((N, N), dtype=int)
for i in range(N):
  for j in range(N):
    if vars_min[i, j].X > 0.5:
      solution[i, j] = 1

G = nx.from_numpy_array(solution, create_using=nx.DiGraph)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=700, edge_color='k', linewidths=1, font_size=15, arrows=True)
plt.show()

# Part 1 1B
model_max = Model()
vars_max = model_max.addVars(N, N, vtype=GRB.BINARY)
model_max.setObjective(sum(vars_max[i, j] * A1[i, j] for i in range(N) for j in range(N)), GRB.MAXIMIZE)

for i in range(N):
  model_max.addConstr(sum(vars_max[i, j] for j in range(N)) == 1)
  model_max.addConstr(sum(vars_max[j, i] for j in range(N)) == 1)

model_max.optimize()

solution = np.zeros((N, N), dtype=int)
for i in range(N):
  for j in range(N):
    if vars_max[i, j].X > 0.5:
      solution[i, j] = int(vars_max[i, j].X if model_max.status == GRB.OPTIMAL else 0)

G = nx.from_numpy_array(solution, create_using=nx.DiGraph)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=700, edge_color='k', linewidths=1, font_size=15, arrows=True)
plt.show()

# Part 1 2
model_2_way = Model()
vars_2_way = model_2_way.addVars(N, N, vtype=GRB.BINARY)
model_2_way.setObjective(gp.quicksum(vars_2_way[i, j] * A2[i, j] * A2[j, i] for i in range(N) for j in range(i+1, N)), GRB.MAXIMIZE)

for i in range(N):
  model_2_way.addConstr(gp.quicksum(vars_2_way[i, j] + vars_2_way[j, i] for j in range(N) if j != i) <= 1)

model_2_way.optimize()

solution = np.zeros((N, N), dtype=int)
for i in range(N):
  for j in range(N):
    if vars_2_way[i, j].X > 0.5 and A2[i, j] == 1 and A2[j, i] == 1:
      solution[i, j] = solution[j, i] = 1

G = nx.from_numpy_array(solution, create_using=nx.DiGraph)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=700, edge_color='k', linewidths=1, font_size=15, arrows=True)
plt.show()

# Part 2
A1_compatible = np.where(A1 < 11, 1, 0)
df_A1_compatible = pd.DataFrame(A1_compatible)
print(df_A1_compatible)
model_priorities = Model()
vars_priorities = model_priorities.addVars(N, N, vtype=GRB.BINARY)
model_priorities.setObjective(gp.quicksum(vars_priorities[i, j] for i in range(N) for j in range(N)), GRB.MAXIMIZE)

for i in range(N):
  model_priorities.addConstr(gp.quicksum(vars_priorities[i, j] for j in range(N)) <= 1)
  model_priorities.addConstr(gp.quicksum(vars_priorities[j, i] for j in range(N)) <= 1)

for i in range(N):
    for j in range(N):
        model_priorities.addConstr(vars_priorities[i, j] <= A1_compatible[i, j])
        model_priorities.addConstr(vars_priorities[i, j] == vars_priorities[j, i])


model_priorities.optimize()

solution = np.zeros((N, N), dtype=int)
for i in range(N):
  for j in range(N):
    if vars_priorities[i, j].X > 0.5:  # If the exchange is selected in the solution
      solution[i, j] = 1

G = nx.from_numpy_array(solution, create_using=nx.DiGraph)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=700, edge_color='k', linewidths=1, font_size=15, arrows=True)
plt.show()
